In [1]:
#!python -m spacy download es_core_news_sm

In [2]:
import re
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# Build the document ai client
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_info(creds)

from google.cloud import documentai_v1 as documentai
documentai_client = documentai.DocumentProcessorServiceClient(credentials=credentials)

def process_document(image_path):

    file_extension = image_path.split(".")[-1]

    if file_extension == "jpg" or file_extension == "jpeg":
        mime_type = "image/jpeg"
    elif file_extension == "png":
        mime_type = "image/png"
    elif file_extension == "pdf":
        mime_type = "application/pdf"
    else:
        return {
            "text":""
        }

    with open(image_path, "rb") as image:
        image_content = image.read()

    raw_document = documentai.RawDocument(
            content=image_content,
            mime_type=mime_type,
        )

    request = documentai.ProcessRequest(
        #name='projects/212510393549/locations/us/processors/fedf26f965907f56',
        name='projects/239799364293/locations/us/processors/3fb86e3f5c1eb3fc',
        raw_document=raw_document)

    result = documentai_client.process_document(request=request)

    document = result.document
    text = document.text

    return {
        "text":text
    }

In [5]:
from openai import OpenAI
from pydantic import BaseModel

# Función para realizar la llamada a la API de OpenAI con system instructions
def synthesize_text(text):

    client = OpenAI()

    system_message = """
    Eres un experto en editar documentos científicos y tu objetivo es combinar varios textos en uno solo, sin perder absolutamente ningún detalle relevante. 

    Instrucciones específicas:
    1. No debes omitir ningún dato, incluso si parece repetitivo. Todos los detalles técnicos, cifras, y datos mencionados deben ser preservados.
    2. Organiza el texto en secciones claras: Introducción, Objetivo, Metodología, Resultados y Conclusiones. Asegúrate de que todas las cifras y resultados estén claramente asociados con sus métodos experimentales correspondientes.
    3. Si hay información similar entre los textos, combínala de manera fluida, pero asegúrate de que cada aspecto del estudio esté representado en el texto final.
    4. No introduzcas suposiciones, todo el contenido debe derivarse directamente de los textos originales.
    5. Mantén un tono formal y preciso, y asegura la coherencia entre las secciones.
    6. El texto debe ser EXTREMADAMENTE detallado como para que cualquiera que lo lea pueda entender todos los aspectos importantes del estudio, incluidos los resultados clave, los métodos experimentales y las conclusiones.
    7. Todo el contenido relevante que no entre dentro de las categorías especificadas, puedes incluirlo en un apartado de Observaciones.
    IMPORTANTE: No elimines información durante el proceso de síntesis. Puedes re-estructurarla, pero no eliminarla.
    IMPORTANTE: Necesito que incluyas ABSOLUTAMENTE TODAS las cifras que dan soporte a los resultados y las conclusiones, así como las cifras que dan contexto.
    IMPORTANTE: No me interesa que resumas el contenido de los textos, sólo que lo estructures adecuadamente.
    IMPORTANTE: Debes ser específico, trata de no realizar generalizaciones. Prefiero que respondas con detalles.
    Incluye el nombre original del trabajo además de su traducción al español.
    Divide las diferentes secciones con ## de Markdown, porque luego lo usaré para splitting.
    """


    # Prompt que será enviado al modelo junto con los textos
    prompt = (
        f"""Dado los siguientes textos provenientes de un proceso de OCR en varios archivos relativos a un trabajo de investigación, genera un solo texto que sintetice toda la información de manera clara y concisa, siguiendo las instrucciones del sistema.
        Textos: {text},
        """
    )
    

    class PaperSynthesis(BaseModel):
        paper_title: str
        synthesis: str



    # Realizar la llamada a la API de Open
        # Realizar la llamada a la API de OpenAI
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"{system_message}"},
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ],
        seed=42,
        max_tokens=10000,
        temperature=0.05,
        top_p=1,
        n=1,
        response_format=PaperSynthesis
    )
    

    # Extraer la respuesta generada
    synthesized_text = completion.choices[0].message.parsed
    return synthesized_text

In [6]:
def generate_chunks(research_paper_title,context,text):

   client = OpenAI()

   system_instructions = """
   Eres un experto en la creación de fragmentos optimizados (chunks) para un proceso de Recuperación Augmentada por Generación (RAG). 
   Te proporcionaremos tres elementos:
   1. El texto completo del trabajo de investigación, que podrás utilizar para obtener contexto adicional.
   2. El nombre del trabajo de investigación del cual proviene dicho texto.
   3. Un fragmento específico del texto original.

   Tu tarea será dividir el fragmento proporcionado en uno o más chunks con las siguientes condiciones:
   1. Cada chunk debe tener entre 100 y 150 tokens.
   2. Cada chunk debe empezar con un encabezado que contenga:
      - El nombre del trabajo de investigación.
      - Un breve resumen del contexto del trabajo.

   El formato del encabezado será el siguiente:
   "El siguiente es un fragmento del trabajo: <nombre del trabajo de investigación>, que trata sobre <resumen del contexto>: <chunk>"

   Consideraciones importantes:
   - Los chunks deben ser coherentes por sí mismos y formar una unidad semántica completa.
   - Asegúrate de que cada chunk preserve tanto el contexto relevante como la información clave del texto original.
   - No omitas información importante ni repitas contenido innecesariamente entre los chunks.

   Tu objetivo es garantizar que cada fragmento sea comprensible y mantenga el propósito del texto original.
   IMPORTANTE: NO ELIMINES INFORMACIÓN AL REALIZAR LOS CHUNKS
   IMPORTANTE: Debes ser específico, trata de no realizar generalizaciones. Prefiero que respondas con detalles.
   """

   user_message = f"""
   Texto completo del trabajo de investigación: "{context}"

   Nombre del trabajo de investigación: "{research_paper_title}"

   Fragmento específico del texto original: "{text}"

   Tu tarea es dividir el fragmento proporcionado en chunks de 100 a 120 tokens siguiendo las indicaciones anteriores. 
   Cada chunk debe incluir un encabezado con el nombre del trabajo y un breve resumen del contexto. Asegúrate de que cada chunk sea coherente y forme una unidad semántica completa.
   """

   class SynthesisChunks(BaseModel):
      paper_title: str
      chunks: list[str]



   # Realizar la llamada a la API de Open
      # Realizar la llamada a la API de OpenAI
   completion = client.beta.chat.completions.parse(
      model="gpt-4o",
      messages=[
         {
               "role": "system",
               "content": f"{system_instructions}"},
         {
               "role": "user",
               "content": f"{user_message}"
         }
      ],
      seed=42,
      max_tokens=10000,
      temperature=0.01,
      top_p=1,
      n=1,
      response_format=SynthesisChunks
   )


   # Extraer la respuesta generada
   parsed_chunks = completion.choices[0].message.parsed

   return parsed_chunks


In [7]:
import io
import time
from openai import OpenAI

client = OpenAI()

def upload_chunk(filename,chunk):
    # Convertir el chunk a binario
    binary_db_chunk = chunk.encode('utf-8')

    # Crear un buffer binario
    buffer = io.BytesIO(binary_db_chunk)

    try:
        # Subir el archivo a OpenAI
        openai_file = client.files.create(
            file=(filename, buffer, "text/plain"),
            purpose="assistants"
        )

        # Intentar hasta 10 veces que el archivo esté procesado
        for i in range(10):
            # Verificar si el archivo está procesado
            if openai_file.status == 'processed':
                # Subir el archivo a vector store
                vector_store_file = client.beta.vector_stores.files.create(
                    vector_store_id="vs_UJzqALumjqTk5ZzN18k4GrjA",
                    file_id=openai_file.id,
                    chunking_strategy={
                        "type":"static",
                        "static":{
                            "max_chunk_size_tokens":200,
                            "chunk_overlap_tokens":50
                        }}
                )
                return vector_store_file  # Regresar el archivo subido
            else:
                time.sleep(0.2)  # Esperar antes de reintentar

        raise TimeoutError("El archivo no fue procesado en el tiempo esperado.")
    
    finally:
        # Cerrar el buffer al finalizar
        buffer.close()



# LOOP

In [9]:
from tqdm import tqdm

# Define the root directory (in this case 'data')
root_directory = '../data' ### RESTORE THIS FOLDER

# Get the list of subfolders in the root directory
subfolders = [f.path for f in os.scandir(root_directory) if f.is_dir() and f.name != 'Preguntas orientadoras']

# Function to walk through a subfolder and return all files
def get_files_in_subfolder(subfolder):
    file_list = []
    for root, dirs, files in os.walk(subfolder):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list


db_chunks = []

# Iterate over each subfolder and print the list of files in it
for subfolder in tqdm(subfolders[11:], desc="Processing subfolders"):

    subfolder_content = {}
    files_in_subfolder = get_files_in_subfolder(subfolder)
    
    for i,file in enumerate(files_in_subfolder):
        filename = file.split("/")[-1]
        try:
            text = process_document(file)
            content = {
                "filename":filename,
                "ocr_text":text["text"]}
            subfolder_content[str(i)]=content
        except Exception as e:
            print(f"Error processing file {file}: {str(e)}")

    # Once the folder content is obtained, send all data to an LLM for synthesis.
    synthesized_output = synthesize_text(subfolder_content)
    synthesized_dict = synthesized_output.json() # Convert pydantic object into a dictionary

    # Almacenar los diccionarios con la síntesis de cada carpeta en un archivo de texto que será utilizado como msg.
    content = str(synthesized_dict)
    file_path = '../research_studies.txt'
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write(content + '\n')

    # Sintetizar chunks que contengan información de contexto
    research_paper_title = eval(synthesized_dict)['paper_title']
    context = eval(synthesized_dict)['synthesis']
    
    splitted_text = re.split(r'(?<!#)##(?!#)', context) # Se utilizan las marcas de sub titulo markdown para el splitting, ya que cada marca va por sección.
    splitted_text = list(filter(lambda x: len(x.strip()) > 0, splitted_text))

    # Almacenar los diferentes chunks en una lista para luego incluirlos en un vector store
    for text in splitted_text[:]:
        synthetic_chunks = generate_chunks(research_paper_title,context,text)
        chunks = eval(synthetic_chunks.json())['chunks']
        db_chunks.extend(chunks)

Processing subfolders:   0%|          | 0/14 [00:00<?, ?it/s]

Error processing file ../data/8/PPT-Niti_et_al_Performance_Immune-_Response_White-Shrimp_.Litopenaeus-vannamei._EN--.pdf: 400 Document pages exceed the limit: 15 got 24 [reason: "PAGE_LIMIT_EXCEEDED"
domain: "documentai.googleapis.com"
metadata {
  key: "pages"
  value: "24"
}
metadata {
  key: "page_limit"
  value: "15"
}
]


Processing subfolders: 100%|██████████| 14/14 [11:00<00:00, 47.16s/it]


In [10]:
# Una vez creados los chunks, súbelos al vector store
for index, db_chunk in enumerate(tqdm(db_chunks[:], desc="Uploading chunks to vector store")):

    try:
        # Subir cada chunk al vector store
        vector_store_file = upload_chunk(filename=f"file_{str(index)}.txt", chunk=db_chunk)
        time.sleep(0.2)  # Esperar antes de continuar para evitar sobre cargar la api

    except Exception as e:
        print(f"Error al subir el chunk {index}: {e}")

Uploading chunks to vector store: 100%|██████████| 170/170 [04:10<00:00,  1.47s/it]


In [ ]:
# Una vez creados los chunks, súbelos al vector store
for index, db_chunk in enumerate(tqdm(db_chunks[:], desc="Uploading chunks to vector store")):

    try:
        # Subir cada chunk al vector store
        vector_store_file = upload_chunk(filename=f"file_{str(index)}.txt", chunk=db_chunk)
        time.sleep(0.2)  # Esperar antes de continuar para evitar sobre cargar la api

    except Exception as e:
        print(f"Error al subir el chunk {index}: {e}")

In [3]:
# Definimos la variable global para los datos históricos
historic_data = ""
# Intentamos cargar el archivo al importar el módulo
try:
    with open("../research_studies.txt", "r", encoding="utf-8") as file:
        historic_data = file.read()
except FileNotFoundError:
    print("El archivo research_studies.txt no se encontró.")
except Exception as e:
    print(f"Error al leer el archivo: {e}")

In [12]:
import json

# Path to your txt file
file_path = "../research_studies.txt"

# Open and read the file line by line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Parse each line as a dictionary
            data_dict = json.loads(line)
            # Perform your processing here
            file_name = data_dict['paper_title']
            content = data_dict['synthesis']
            print(data_dict)  # Example print statement to see the dictionary
        except json.JSONDecodeError as e:
            print(f"Error parsing line: {line}")
            print(f"Error message: {e}")


{'paper_title': 'Effect of Sangrovit® Supplementation in Combination with Organic Acids on Stress Response and Diarrhea of Post-Weaning Pigs', 'synthesis': '## Introducción\nEl destete es un evento muy estresante para los lechones debido a un cambio en la dieta, un nuevo entorno y la mezcla con cerdos desconocidos. Además, su sistema inmunológico aún no está completamente desarrollado, lo que hace que la diarrea post-destete (PWD), causada principalmente por E. coli enterotoxigénico (ETEC), sea un problema importante en los lechones destetados. La hipótesis del estudio fue que el uso de Sangrovit® en combinación con una mezcla de ácidos orgánicos podría mitigar el impacto negativo del estrés del destete, mejorar las puntuaciones fecales de los lechones post-destete y reducir la necesidad de tratamientos antibióticos.\n\n## Objetivo\nEvaluar el efecto de la suplementación con Sangrovit® en combinación con ácidos orgánicos sobre la respuesta al estrés y la incidencia de diarrea en lechon